# Diamond Company Analytics Challenge

**Time:** 35 minutes

**Scenario:** You are the new data analyst at a prestigious diamond company. The CEO has called an urgent meeting and needs answers to critical business questions about the inventory.

**Your Mission:** Analyze the diamond inventory data and provide insights to guide strategic decisions.

In [2]:
import polars as pl

# Load the diamonds dataset
print("Loading diamonds dataset...")
df = pl.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv')
print("Dataset loaded successfully!")

Loading diamonds dataset...
Dataset loaded successfully!


## Section 1: Initial Inventory Assessment (5 minutes)

The CEO walks in: "Before we dive into specifics, I need to understand what we're working with. Give me the basics about our inventory database."

### Question 1: "How extensive is our inventory database?"

The CEO needs to know the scale of data you're analyzing.

In [3]:
# Your code here:

print("Our data base has",(df.height),"rows and",(df.width),"columns.")

Our data base has 6000 rows and 8 columns.


### Question 2: "What information do we track about each diamond?"

List all the characteristics we record.

In [4]:
# Your code here:
print("Our data base has the following columns:",(df.columns))

Our data base has the following columns: ['Carat Weight', 'Cut', 'Color', 'Clarity', 'Polish', 'Symmetry', 'Report', 'Price']


### Question 3: "When I look at one line in this database, what am I looking at?"

Explain in business terms:

**Your answer:** Each row represents: a unique diamond

## Section 2: Strategic Business Questions (20 minutes)

The CEO continues: "Now let's get to the important questions. Our board meeting is tomorrow and I need concrete numbers."

### Question 4: "We're considering expanding our storage facilities. How many total carats do we have in inventory? I need to know if our vaults can handle the weight."

Think about what measurement would help answer this.

In [5]:
# Your code here:
total_carats = df.select(pl.col("Carat Weight").sum())
total_carats = total_carats.item()
print(f"We currently have {total_carats:,.0f} carats.")


We currently have 8,007 carats.


### Question 5: "Our new marketing campaign claims we have 'the widest variety of cuts in the industry.' Can you verify this?"

The CEO wants to know about variety, not quantity.

In [13]:
# Your code here:
unique_cuts = df.select(pl.col("Cut")).unique().height
print("We have",(unique_cuts),"variaties of cuts: ",(df.select(pl.col("Cut")).unique()).to_series().to_list())


We have 5 variaties of cuts:  ['Fair', 'Signature-Ideal', 'Good', 'Ideal', 'Very Good']


### Question 6: "I'm meeting with our insurance company. They need to know our price exposure - what's our most and least valuable diamond?"

In [28]:
# Your code here:
max_price = df.select(pl.col("Price")).max()
min_price = df.select(pl.col("Price")).min()

max_price = max_price.item()
min_price = min_price.item()

print(f"The most valuable diamond is worth ${max_price:,.2f}.")
print(f"The least valuable diamond is worth ${min_price:,.2f}.")

The most valuable diamond is worth $101,561.00.
The least valuable diamond is worth $2,184.00.


### Question 7: "Which color grades should we focus on in our premium line? I need typical prices across our color spectrum."

The CEO wants to understand pricing patterns.

In [43]:
# Your code here:
avg_price_per_color = df.group_by("Color").agg(pl.col("Price").mean().round(2).alias("Average Price"))
avg_price_per_color = avg_price_per_color.sort("Average Price",descending=True)
print(avg_price_per_color)

shape: (6, 2)
┌───────┬───────────────┐
│ Color ┆ Average Price │
│ ---   ┆ ---           │
│ str   ┆ f64           │
╞═══════╪═══════════════╡
│ D     ┆ 15255.78      │
│ F     ┆ 12712.24      │
│ G     ┆ 12520.05      │
│ E     ┆ 11539.19      │
│ H     ┆ 10487.35      │
│ I     ┆ 8989.64       │
└───────┴───────────────┘


**Reflection:** After analyzing by color, how has your view of the data changed? What does each row in your result represent now?

**Your answer:** New grain = unique colors of a diamond

## Section 3: Advanced Strategic Analysis (10 minutes)

The CEO leans forward: "These next questions will shape our strategy for next year."

### Question 8: "We're thinking of specializing in certain clarity grades. For our investor deck, I need a complete breakdown showing: how many diamonds we have in each clarity grade, their typical price, and the total carats we're holding for each grade."

In [8]:
# Your code here:
clarity_grades = df.group_by("Clarity").agg(
    pl.col("Price").mean().round(2).alias("Average Price"),
    pl.col("Carat Weight").sum().round(2).alias("Total Carats"),
    pl.count().alias("Number of Diamonds")
)
clarity_grades = clarity_grades.sort("Average Price",descending=True)
print(clarity_grades)

shape: (7, 4)
┌─────────┬───────────────┬──────────────┬────────────────────┐
│ Clarity ┆ Average Price ┆ Total Carats ┆ Number of Diamonds │
│ ---     ┆ ---           ┆ ---          ┆ ---                │
│ str     ┆ f64           ┆ f64          ┆ u32                │
╞═════════╪═══════════════╪══════════════╪════════════════════╡
│ FL      ┆ 63776.0       ┆ 7.87         ┆ 4                  │
│ IF      ┆ 22105.84      ┆ 316.0        ┆ 219                │
│ VVS1    ┆ 16845.68      ┆ 389.43       ┆ 285                │
│ VVS2    ┆ 14142.18      ┆ 876.58       ┆ 666                │
│ VS1     ┆ 13694.11      ┆ 1682.74      ┆ 1192               │
│ VS2     ┆ 11809.05      ┆ 2170.81      ┆ 1575               │
│ SI1     ┆ 8018.86       ┆ 2563.69      ┆ 2059               │
└─────────┴───────────────┴──────────────┴────────────────────┘


/tmp/ipython-input-2368881381.py:5: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("Number of Diamonds")


### Question 9: "My luxury competitor claims they have the best prices for every cut-color combination. Show me our pricing structure across these combinations - I want to see where we're competitive."

In [9]:
# Your code here:
cut_color_price = df.group_by(["Cut", "Color"]).agg(
    pl.col("Price").mean().round(2).alias("Average Price")
)
cut_color_price = cut_color_price.sort("Average Price",descending=True)
print(cut_color_price)

shape: (30, 3)
┌─────────────────┬───────┬───────────────┐
│ Cut             ┆ Color ┆ Average Price │
│ ---             ┆ ---   ┆ ---           │
│ str             ┆ str   ┆ f64           │
╞═════════════════╪═══════╪═══════════════╡
│ Signature-Ideal ┆ D     ┆ 19823.1       │
│ Ideal           ┆ D     ┆ 18461.95      │
│ Ideal           ┆ F     ┆ 14729.43      │
│ Ideal           ┆ G     ┆ 13570.31      │
│ Signature-Ideal ┆ F     ┆ 13247.95      │
│ …               ┆ …     ┆ …             │
│ Fair            ┆ F     ┆ 6063.63       │
│ Fair            ┆ D     ┆ 6058.25       │
│ Fair            ┆ H     ┆ 5908.5        │
│ Fair            ┆ E     ┆ 5370.63       │
│ Fair            ┆ I     ┆ 4573.19       │
└─────────────────┴───────┴───────────────┘


**Reflection:** How specific is your analysis now? What level of detail does each row represent?

**Your answer:** New grain = unique cut AND color combinations of diamonds

### Question 10: URGENT - "Our biggest client is interested in purchasing entire inventory segments. They asked: 'Which color-cut combinations represent your highest total inventory value?' Show me the top 5 most valuable inventory segments by total worth, and include how many diamonds are in each segment."

This is critical for tomorrow's negotiation.

In [15]:
# Your code here:
unique_color_cut = df.group_by(["Color", "Cut"]).agg(
    pl.col("Price").mean().round(2).alias("Average Price"),
    pl.count().alias("Number of Diamonds")
)
unique_color_cut = unique_color_cut.sort("Average Price",descending=True).head(5)
print(unique_color_cut)

##count

shape: (5, 4)
┌───────┬─────────────────┬───────────────┬────────────────────┐
│ Color ┆ Cut             ┆ Average Price ┆ Number of Diamonds │
│ ---   ┆ ---             ┆ ---           ┆ ---                │
│ str   ┆ str             ┆ f64           ┆ u32                │
╞═══════╪═════════════════╪═══════════════╪════════════════════╡
│ D     ┆ Signature-Ideal ┆ 19823.1       ┆ 30                 │
│ D     ┆ Ideal           ┆ 18461.95      ┆ 280                │
│ F     ┆ Ideal           ┆ 14729.43      ┆ 363                │
│ G     ┆ Ideal           ┆ 13570.31      ┆ 690                │
│ F     ┆ Signature-Ideal ┆ 13247.95      ┆ 38                 │
└───────┴─────────────────┴───────────────┴────────────────────┘


/tmp/ipython-input-3293466298.py:4: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("Number of Diamonds")


## Debrief

Congratulations! You've just completed your first day as the diamond company's data analyst.

Remember:
- CEOs speak in business language, not technical terms
- Always understand what each row represents before analyzing
- Business questions contain clues about which operations to use